In [1]:
import pandas
import numpy
df_raw = pandas.read_json("raw_data.json")

In [2]:
# Variables

reforges = [
    "\u278c",
    "\u25c6",
    "\u278a",
    "\u278e",
    "\u278b",
    "\u273f",
    "\u278d",
    " ✦",
    "⚚ ",
    " ✪",
    "✪",
    "Stiff ",
    "Lucky ",
    "Jerry's ",
    "Dirty ",
    "Fabled ",
    "Suspicious ",
    "Gilded ",
    "Warped ",
    "Withered ",
    "Bulky ",
    "Stellar ",
    "Heated ",
    "Ambered ",
    "Fruitful ",
    "Magnetic ",
    "Fleet ",
    "Mithraic ",
    "Auspicious ",
    "Refined ",
    "Headstrong ",
    "Precise ",
    "Spiritual ",
    "Moil ",
    "Blessed ",
    "Toil ",
    "Bountiful ",
    "Candied ",
    "Submerged ",
    "Reinforced ",
    "Cubic ",
    "Warped ",
    "Undead ",
    "Ridiculous ",
    "Necrotic ",
    "Spiked ",
    "Jaded ",
    "Loving ",
    "Perfect ",
    "Renowned ",
    "Giant ",
    "Empowered ",
    "Ancient ",
    "Sweet ",
    "Silky ",
    "Bloody ",
    "Shaded ",
    "Gentle ",
    "Odd ",
    "Fast ",
    "Fair ",
    "Epic ",
    "Sharp ",
    "Heroic ",
    "Spicy ",
    "Legendary ",
    "Deadly ",
    "Fine ",
    "Grand ",
    "Hasty ",
    "Neat ",
    "Rapid ",
    "Unreal ",
    "Awkward ",
    "Rich ",
    "Clean ",
    "Fierce ",
    "Heavy ",
    "Light ",
    "Mythic ",
    "Pure ",
    "Smart ",
    "Titanic ",
    "Wise ",
    "Bizarre ",
    "Itchy ",
    "Ominous ",
    "Pleasant ",
    "Pretty ",
    "Shiny ",
    "Simple ",
    "Strange ",
    "Vivid ",
    "Godly ",
    "Demonic ",
    "Forceful ",
    "Hurtful ",
    "Keen ",
    "Strong ",
    "Superior ",
    "Unpleasant ",
    "Zealous ",
]

df = df_raw.query("bin == False")
df = df[["uuid","item_name","starting_bid","tier"]][~df["item_lore"].str.contains(r"\bfurniture\b")]
df["item_id"] = df["item_name"].str.replace('|'.join(reforges), '', regex=True).replace(r"\[[^\]]*\] ", "", regex=True).replace(r'^\s+', '', regex=True)
df["item_id"] = df['item_id'].astype(str) + '|' + df['tier'].astype(str)
df = df.groupby('item_id').agg({'uuid': lambda x: list(x), 'item_name': lambda x: list(x), 'starting_bid': lambda x: list(x), 'tier': lambda x: list(x)})
df = df[df['tier'].str.len() >= 2]


In [3]:
items = pandas.DataFrame(columns=["item_name", "uuid", "price", "tier", "profit", "profit_percent", "mean", "median", "std"])
items_index = 0


for index, row in df.iterrows():

    prices = row["starting_bid"]
    modal_index = prices.index(min(prices))
    prices = sorted(prices)

    price = prices[0]
    profit = prices[1] - price
    profit_percent = (profit / price) * 100

    if profit < 500: continue

    uuid = row["uuid"][modal_index]
    tier = row["tier"][modal_index]
    item_name = row["item_name"][modal_index]

    count = len(prices)
    mean = numpy.mean(prices).astype(int)
    std = numpy.std(prices).astype(int)
    median = numpy.median(prices).astype(int)

    items.loc[items_index] = {
        "uuid": uuid,
        "item_name": item_name,
        "price": price,
        "tier": tier,
        "profit": profit,
        "profit_percent": profit_percent,
        "mean": mean,
        "median": median,
        "std": std,
    }

    items_index += 1

In [13]:
srt_items = items[(items["price"] >= 500) & (items["price"] <= 10000000) & (items["profit"] >= 1) & (items["profit"] <= 10000000)]

srt_items = srt_items.sort_values("profit",ignore_index=True)
print(srt_items.to_markdown())

|     | item_name                              | uuid                             |    price | tier      |   profit |   profit_percent |     mean |   median |      std |
|----:|:---------------------------------------|:---------------------------------|---------:|:----------|---------:|-----------------:|---------:|---------:|---------:|
|   0 | Bonzo's Staff                          | 8e1e54812ad140d793c11012fc134c21 |      500 | RARE      |      500 |     100          |  2431050 |  2149500 |  2242397 |
|   1 | [Lvl 90] Tiger                         | 986e76e6480c43ec8c9ffeb0f4130a53 |      500 | EPIC      |      500 |     100          |  8800300 |  3000000 | 10457280 |
|   2 | Glacite Chestplate                     | edab705451f4453b92f65a0947d106d1 |      500 | EPIC      |      500 |     100          |    45723 |    22500 |    60374 |
|   3 | Miner Leggings                         | c55b4b0667c04a3c84e096b88ad260f3 |     2000 | RARE      |      500 |      25          |    27986 |   

In [5]:
print(len(df_raw))
print(len(df))
print(len(items))
items.to_json("pandas_final.json", indent=4)

57345
738
296
